In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# df1 = pd.read_csv(r'C:\Users\fdiazl\Downloads\Banking Dataset Classification\new_test.csv')
df = pd.read_csv(r'C:\Users\fdiazl\Downloads\Banking Dataset Classification\new_train.csv')
# df = pd.concat([df1, df2], ignore_index=True)

# EDA

In [3]:
df.head(3)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,999,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,999,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,999,0,nonexistent,yes


In [4]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
numerical_columns = list(df.dtypes[df.dtypes != 'object'].index)

for column in categorical_columns:
    print(f'Column: {column}')
    print(df[column].value_counts())
    print('-----------------------')

Column: job
job
admin.           8314
blue-collar      7441
technician       5400
services         3196
management       2345
retired          1366
entrepreneur     1160
self-employed    1099
housemaid         855
unemployed        798
student           711
unknown           265
Name: count, dtype: int64
-----------------------
Column: marital
marital
married     19953
single       9257
divorced     3675
unknown        65
Name: count, dtype: int64
-----------------------
Column: education
education
university.degree      9736
high.school            7596
basic.9y               4826
professional.course    4192
basic.4y               3322
basic.6y               1865
unknown                1397
illiterate               16
Name: count, dtype: int64
-----------------------
Column: default
default
no         26007
unknown     6940
yes            3
Name: count, dtype: int64
-----------------------
Column: housing
housing
yes        17254
no         14900
unknown      796
Name: count, dtype: in

In [26]:
df[categorical_columns].nunique()

job            12
marital         4
education       8
default         3
housing         3
loan            3
contact         2
month          10
day_of_week     5
poutcome        3
dtype: int64

In [5]:
df[numerical_columns].describe()

,age,duration,campaign,pdays,previous
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.014112,258.127466,2.560607,962.052413,0.174719
std,10.403636,258.975917,2.752326,187.951096,0.499025
min,17.000000,0.000000,1.000000,0.000000,0.000000
25%,32.000000,103.000000,1.000000,999.000000,0.000000
50%,38.000000,180.000000,2.000000,999.000000,0.000000
75%,47.000000,319.000000,3.000000,999.000000,0.000000
max,98.000000,4918.000000,56.000000,999.000000,7.000000


In [8]:
# for column in numerical_columns:
#     plt.figure(figsize=(8,4))
#     plt.hist(df[column].dropna(), bins=30, color='blue', alpha=0.7)
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Frequency')
#     plt.grid(axis='y', alpha=0.75)
#     plt.show()

In [ ]:
# data preparation
print(df.isnull().sum())
# print(df.info())

age            0
job            0
marital        0
education      0
default        0
housing        0
loan           0
contact        0
month          0
day_of_week    0
duration       0
campaign       0
pdays          0
previous       0
poutcome       0
y              0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          32950 non-null  int64 
 1   job          32950 non-null  object
 2   marital      32950 non-null  object
 3   education    32950 non-null  object
 4   default      32950 non-null  object
 5   housing      32950 non-null  object
 6   loan         32950 non-null  object
 7   contact      32950 non-null  object
 8   month        32950 non-null  object
 9   day_of_week  32950 non-null  object
 10  duration     32950 non-null  int64 
 11  campaign     32950 non-null  int64 
 12  pdays        32950 non-null

In [ ]:
100*np.round(df['y'].value_counts()/len(df),2)


y
no     89.0
yes    11.0
Name: count, dtype: float64

In [11]:
df['y'] = df['y'].map({'yes': 1, 'no': 0})

In [22]:
x_var = []
y_var = []
correl = []

for x in df[numerical_columns].corr().index:
    for y in df[numerical_columns].corr().columns:
        if x != y:
            x_var.append(x)
            y_var.append(y)
            correl.append(df[numerical_columns].corr().loc[x, y])
correlation_df = pd.DataFrame({'x': x_var, 'y': y_var, 'correlation': correl})
correlation_df = correlation_df.sort_values(by='correlation', ascending=False)
correlation_df.drop_duplicates(subset=['correlation'], inplace=True)
correlation_df.reset_index(drop=True, inplace=True)
correlation_df.head(15)

,x,y,correlation
0,campaign,pdays,0.053795
1,previous,duration,0.022538
2,age,previous,0.020670
3,campaign,age,0.003302
4,age,duration,-0.001841
5,pdays,age,-0.032011
6,duration,pdays,-0.047127
7,campaign,duration,-0.075663
8,campaign,previous,-0.079051
9,pdays,previous,-0.589601


In [48]:
# train test split
from sklearn.model_selection import train_test_split
    
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [53]:
from sklearn.metrics import mutual_info_score

# categorical_columns.remove('y')

for c in categorical_columns:
    print(c, round(mutual_info_score(df_full_train['y'], df_full_train[c]),2))

job 0.01
marital 0.0
education 0.0
default 0.01
housing 0.0
loan 0.0
contact 0.01
month 0.03
day_of_week 0.0
poutcome 0.03


In [54]:
print("Long term deposit rate general: ", np.round(df_full_train.y.mean(),2))
print("-------------------------------")
print("Long term deposit rate for unknown default: ", np.round(df_full_train[df_full_train['default']=='unknown'].y.mean(),2))
print("Long term deposit rate for no default: ", np.round(df_full_train[df_full_train['default']=='no'].y.mean(),2))
print("Long term deposit rate for yes default: ", np.round(df_full_train[df_full_train['default']=='yes'].y.mean(),2))
print("-------------------------------")
print("Long term deposit rate for failure: ", np.round(df_full_train[df_full_train['poutcome']=='failure'].y.mean(),2))
print("Long term deposit rate for success: ", np.round(df_full_train[df_full_train['poutcome']=='success'].y.mean(),2))
print("Long term deposit rate for nonexistent: ", np.round(df_full_train[df_full_train['poutcome']=='nonexistent'].y.mean(),2))
print("-------------------------------")
print("Long term deposit rate for loan: ", np.round(df_full_train[df_full_train['loan']=='yes'].y.mean(),2))
print("Long term deposit rate for no loan: ", np.round(df_full_train[df_full_train['loan']=='no'].y.mean(),2))
print("Long term deposit rate for unknown loan: ", np.round(df_full_train[df_full_train['loan']=='unknown'].y.mean(),2))


Long term deposit rate general:  0.11
-------------------------------
Long term deposit rate for unknown default:  0.05
Long term deposit rate for no default:  0.13
Long term deposit rate for yes default:  0.0
-------------------------------
Long term deposit rate for failure:  0.14
Long term deposit rate for success:  0.64
Long term deposit rate for nonexistent:  0.09
-------------------------------
Long term deposit rate for loan:  0.11
Long term deposit rate for no loan:  0.11
Long term deposit rate for unknown loan:  0.11


In [50]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

df_train.drop(['y', 'pdays'], axis=1, inplace=True)
df_val.drop(['y', 'pdays'], axis=1, inplace=True)
df_test.drop(['y', 'pdays'], axis=1, inplace=True)

In [52]:
# entrenaimiento
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

dv = DictVectorizer(sparse=False)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

def model_regression(df_train, y_train, C=1.0):
    dv = DictVectorizer(sparse=False)
    train_dicts = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    return dv, model

def evaluate_model(dv, model, df_val, y_val):
    
    val_dicts = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dicts)

    y_pred = model.predict_proba(X_val)[:, 1]
    churn_decision = (y_pred >= 0.5)
    accuracy = (churn_decision == y_val).mean()
    
    return accuracy